In [1]:
# Data di questa revisione

! date

Sun Jun 28 00:18:24 CEST 2020


# Un interprete per il linguaggio Tiny Hi

Il progetto richiede l'implementazione di un interprete per una versione semplificata del linguaggio *Hi*, denominata *Tiny Hi*, sviluppata da Martin Buchanan nel 1976 e descritta in un [articolo](TinyHi.pdf) del *Dr. Dobb's Journal of Computer Calisthenics* del medesimo anno.

A tal fine:

* deve essere realizzata una **grammatica** derivante dalle specifiche illustrate di seguito (in parte differenti da quelle pubblicate nel 1976);
* deve essere implementato un **parser**, costruito manualmente, o utilizzando ANTLR;
* deve essere implementato un **interprete**, di tipo ricorsivo, o iterativo (basato su *code threading*).

Riguardo al primo punto, si presti attenzione al fatto che *la grammatica definita nell'articolo presenta diverse criticità* (tra le quali, il fatto che è ambigua), ragion per cui riscriverla interamente potrebbe essere un'ottima strategia.

## Aggiornamenti

Questo documento contiene degli aggiornamenti rispetto alla prima versione pubblicata; le parti rimosse sono idicate da <del>una riga che attraversa il testo</del> mentre le parti nuove, o a cui è necessario prestare rinnovata attenzione, sono indicate da <mark>una sottolineatura gialla</mark>. Chi usasse un visualzzatore di *notebook* che non mostra tali segni può cercare, rispettivamente, gli elementi `<del>` e `<mark>` nel testo. 

## Modalità di sviluppo e consegna

Il codice deve essere consegnato tramite un **apposito repository** su GitHub creato dal docente su richiesta degli studenti. Gli studenti sono incoraggiati ad effettuare frequenti **commit** periodici per testimoniare il processo di sviluppo e sono altresì incoraggiati ad effettuare (anche con una frequenza ridotta rispetto ai *commit*) dei **push** periodici al fine di informare il docente dello stato di avanzamento del progetto (cosa che rende possibili, anche se non garantite, delle eventuali *code review* del codice da parte del docente). Al termine del lavoro, con congruo anticipo rispetto alla data del colloquio, lo studente effettua un commit con messaggio `CONSEGNA FINALE` che segna il termine del processo di sviluppo e l'inizio della valutazione da parte del docente. Chi non è in grado di usare `git` e GitHub può documentarsi per tempo in rete su come usare tali strumenti.

Non è richiesta la stesura di una *relazione*; per questa ragione è indicato che il codice contenga un **documentazione** sufficiente a guidare il docente nella comprensione della sua architettura, delle scelte progettuali ed implementative e del comportamento delle sue varie parti. Si consiglia di redigere tale documentazione sotto forma di *commenti nel codice*, usando eventualmente (solo se si è in grado e senza perdere tempo) uno degli strumenti di documentazione automatica disponibili per il linguaggio scelto.

Il codice deve consentire un ragionevole **testing**, se non altro sotto forma di test di accettazione sviluppati a partire dai sorgenti forniti in calce a questa documentazione. Siete liberi di implementare i vostri test, o di usare il file [tests.py](tests.py) messo a disposizione del docente.

<mark>Si suggerisce di raccogliere il codice in uno o più moduli (file `.py`) Python, dato che deve essere possibile eseguire test in modo automatico da parte del docente, il codice dell'interprete non deve essere contenuto in un *notebook* Python.</mark>

## Specifiche del linguaggio Tiny Hi

Le specifiche di massima del linguaggio a cui fare riferimento nello svolgimento del progetto sono basate sull'[articolo originale](TinyHi.pdf). Rispetto alla descrizione originale, valogono le seguenti modifiche e/o precisazioni.

### Tokenizzazione e parsing

Le specifiche originali non fanno menzione del *white-space*, potete assumere che esso sia ignorato — ad eccezione del *newline* utile a terminare <del>alcune specifiche</del> <mark>le</mark> istruzioni. 

Una particolare difficoltà è introdotta dalla scelta del linguaggio di usare uno spazio per indicare la concatenazione tra espressioni intere (che danno luogo a vettori) e stringa (che risultano in una stringa); l'"operatore" spazio definito nell'articolo ha priorità molto alta, la scelta di ignorare i *white-space* impone perciò particolare cura nel progettare grammatica e parser perché sia rispettata la semantica del linguaggio secondo cui, ad esempio, l'espressione `1 2 + 3 4` non ha per valore il vettore di tre componenti `1 5 4`, bensì il vettore di due componenti `4 6`.

Un altro elemento critico è l'uso del token `-` (in particolare, in rapporto ai vettori). Esso può essere usato per indicare sia i numeri negativi, come ad esempio `-2`, che l'opertore unario di negazione, come ad esempio `- 1 2` (che equivale al vettore di due componenti `-1 -2`, dato che la concatenazione ha precedenza maggiore della negazione) e l'operatore binario di sottrazione, come ad esempio `2 - 1` (che equivale a `1`). Al fine di semplificare la realizzazione del progetto potete assumere che nei numeri negativi il segno `-` non sia mai separato (con *white-space*) dal numero (ossia che si scriva `-1`, non `- 1`) e che l'operatore unario di negazione sia rimpiazzato da `~` (al posto del segno `-`).

Un ultimo punto critico, sempre in rapporto ai vettori, sono le funzioni (ad un solo argomento): l'espressione `F (1)` può infatti essere intesa come il vettore a due componenti `F`  e `1` (racchiuso tra parentesi), che come l'invocazione della funzione `F` con argomento `1`. Sempre al fine di semplificare la realizzazione del progetto potete assumere che nell'invocazione il nome della funzione non sia mai separato (con *white-space*) dalla parentesi tonda aperta; secondo l'esempio di prima `F(1)` è una invocazione di fuzione, mentre `F (1)` è un vettore a due componenti. Stessa cosa vale per la definizione degli eventuali paramentri dopo il `BEGIN`.

Sempre per semplicità può essere completamente ignorata la questione dei *commenti* (e del testo "libero" che segue il token `END`).

In fine, tutti i vincoli sul numero massimo di ripetizioni sono da considerarsi non <del>vincolanti</del> <mark>rilevanti</mark>: è possibile considerare token di lunghezza libera.

### Variabili e blocchi

Le variabili sono *globali* (se il loro nome inizia col punto), oppure *locali* al blocco `BEGIN`/`END` in cui sono definite; alla stessa variabile possono essere assegnati (in momenti diversi dell'esecuzione) valori di tipo diverso (interi, vettori o stringhe). I parametri formali sono locali ai blocchi e il loro valore non può essere modificato; la gestione di variabili locali e parametri avviene tramite *record di attivazione* in modo che la *ricorsione* funzioni come usualmente inteso.

Sebbene i blocchi `BEGIN`/`END` possano essere annidati, per ragioni di semplicità assumeremo viceversa che ci sia un *unico spazio dei nomi* per i blocchi, detto altrimenti: non possono esserci due blocchi con lo stesso nome <mark>e la visibilità di un blocco inizia nel punto in cui è definito e termina al terminare del blocco in cui è definito</mark>. <del>Tuttavia la visibilità di tali nomi è ristretta al blocco entro cui sono definiti questo comporta che un blocco possa essere invocato solo nel corpo del blocco dov'è definito, a qeusta regola fa eccezione il corpo del blocco che può invocare il blocco stesso, questo per consentire la ricorsione</del>.

<mark>Potete in generale non implementare le costanti esterne, in particolare, dal momento che non esiste un tipo *boolean* non è necessario implementare le costanti `.TURE` e `.FALSE`.</mark> 

### Esecuzione

L'esecuzione di un programma corrisponde all'esecuzione delle istruzioni del blocco più esterno (che può eventualmente contenere invocazioni di blocchi definiti nel medesimo).

<mark>Nel linguaggio non è contemplata l'istruzione `RETURN`, per restituire un valore, come viene assegnato un valore al simbolo corrispondente al nome del blocco `BEGIN`/`END`; un esempio è dato dal calcolo del fattoriale nel file `test.py´.</mark>

<mark>La grammatica non specifica esplicitamente se il *brodcasting* (la funzionalità per cui è possibile sommare o moltiplicare un singolo intero per una lista di interi) richiede che l'intero sia a sinistra, o a destra, della lista; potete scegliere se definirlo quale sia l'ordine degli operandi, purché rendiate esplicita la scelta (nei commenti).</mark>

### I/O

Per facilitare il testing, potete assumere che non ci siano istruzioni di *input* (ossia non ci siano mai nomi di variabile preceduti da `?`) e che l'*output* avvenga semplicemente invocando la funzione `print` <mark>di Python</mark>.

## Testing

Il file [tests.py](tests.py) definisce due dizionari `PARSER_TESTS` e `INTERPRETER_TESTS` che contengono, rispettivamente, alcuni test per validare il comportamento del parser e dell'interprete. Tale file contiene anche del codice per svolgere in modo automatico dei test basati su tali dizionari (e delle istruzioni di funzionamento). 

Tali test saranno utilizzati dal docente durante l'orale per valutare il comportamento dell'implementazione presentata.